In [1]:
!pip install requests beautifulsoup4 langchain openai duckduckgo-search pydantic


  Obtaining dependency information for duckduckgo-search from https://files.pythonhosted.org/packages/f5/67/b1eb59a859717043b605eb16d4f3afe46ec3614804ac0fbe594339667866/duckduckgo_search-6.2.11-py3-none-any.whl.metadata
  Obtaining dependency information for click>=8.1.7 from https://files.pythonhosted.org/packages/00/2e/d53fa4befbf2cfa713304affc7ca780ce4fc1fd8710527771b58311a3229/click-8.1.7-py3-none-any.whl.metadata
  Obtaining dependency information for primp>=0.6.1 from https://files.pythonhosted.org/packages/e6/69/c30564f546dc2c1f118526d2f1dc17d8440b2fe2c574018333881450a860/primp-0.6.1-cp38-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/97.9 kB ? eta -:--:--
   ---------------------------------------- 97.9/97.9 kB ? eta 0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ------------- -------------------------- 0.9/2.7 MB 28.7 MB/s eta 0:00:01
   --------------------------------- ------ 2.3/2.7 MB 29.0 MB/s eta 0:00:01
   --

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
python-lsp-black 1.2.1 requires black>=22.3.0, but you have black 0.0 which is incompatible.
spyder 5.4.3 requires jedi<0.19.0,>=0.17.2, but you have jedi 0.19.1 which is incompatible.


In [9]:
!pip install wikipedia


  Using cached wikipedia-1.4.0-py3-none-any.whl


In [10]:
import wikipedia
print(wikipedia.summary("Artificial intelligence"))


Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals. Such machines may be called AIs.
Some high-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); interacting via human speech (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT, Apple Intelligence, and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general applications, often without being called AI because once something be

In [13]:
import os
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper, WikipediaAPIWrapper
from bs4 import BeautifulSoup  # 웹 스크래핑을 위한 BeautifulSoup

# OpenAI 언어 모델 초기화
llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-1106")

# Wikipedia 검색 도구
class WikipediaSearchToolArgsSchema(BaseModel):
    query: str = Field(description="Wikipedia에 대한 검색 쿼리. 예: '양자 물리학'")

class WikipediaSearchTool(BaseTool):
    name = "WikipediaSearch"
    description = "주어진 쿼리와 관련된 정보를 Wikipedia에서 검색합니다."
    
    args_schema: Type[WikipediaSearchToolArgsSchema] = WikipediaSearchToolArgsSchema

    def _run(self, query):
        wiki = WikipediaAPIWrapper()
        return wiki.run(query)

# DuckDuckGo 검색 도구
class DuckDuckGoSearchToolArgsSchema(BaseModel):
    query: str = Field(description="DuckDuckGo에서 검색할 쿼리입니다.")

class DuckDuckGoSearchTool(BaseTool):
    name = "DuckDuckGoSearch"
    description = "DuckDuckGo에서 정보를 검색합니다."
    
    args_schema: Type[DuckDuckGoSearchToolArgsSchema] = DuckDuckGoSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)

# 웹 스크래핑 도구
class WebScrapeArgsSchema(BaseModel):
    url: str = Field(description="텍스트를 스크랩할 웹사이트의 URL입니다.")

class WebScrapeTool(BaseTool):
    name = "WebScrape"
    description = "주어진 URL에서 텍스트를 스크랩하고 추출합니다."
    
    args_schema: Type[WebScrapeArgsSchema] = WebScrapeArgsSchema

    def _run(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        paragraphs = soup.find_all('p')
        content = '\n'.join([p.get_text() for p in paragraphs])
        return content

# 연구 결과를 .txt 파일에 저장하는 도구
class SaveToTextFileArgsSchema(BaseModel):
    content: str = Field(description="저장할 내용.")
    filename: str = Field(default="research_output.txt", description="연구 결과를 저장할 파일 이름.")

class SaveToTextFileTool(BaseTool):
    name = "SaveToTextFile"
    description = ".txt 파일에 연구 결과를 저장합니다."
    
    args_schema: Type[SaveToTextFileArgsSchema] = SaveToTextFileArgsSchema

    def _run(self, content, filename):
        with open(filename, 'w') as f:
            f.write(content)
        return f"연구 내용이 {filename}에 저장되었습니다."

# 도구를 포함한 에이전트 초기화
agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        WikipediaSearchTool(),
        DuckDuckGoSearchTool(),
        WebScrapeTool(),
        SaveToTextFileTool()
    ],
)

# 연구 쿼리 실행
prompt = """
Search XZ backdoor on Wikipedia first, 
if no results, search on DuckDuckGo. 
if there is a result scrap it ans save as .txt file.
"""

agent.invoke(prompt)




> Entering new AgentExecutor chain...

Invoking: `WikipediaSearch` with `{'query': 'XZ backdoor'}`


Page: XZ Utils backdoor
Summary: In February 2024, a malicious backdoor was introduced to the Linux utility xz within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution capabilities on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.
While xz is commonly present in most Linux distributions, at the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions. The backdoor was discovered by the software developer Andres Freund, who announced his findings on 29 March 2024.

Page: XZ Utils
Summary: XZ Utils (previously LZMA Ut

{'input': '\nSearch XZ backdoor on Wikipedia first, \nif no results, search on DuckDuckGo. \nif there is a result scrap it ans save as .txt file.\n',
 'output': 'The information about the XZ backdoor has been successfully saved in a text file named "XZ_Backdoor_Info.txt". If you need further assistance, feel free to ask!'}